# Feature Selection

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
## data ingestion
#import libraries and creating connection with Mongodb
import pymongo
import json
client = pymongo.MongoClient("localhost")
print(client)
#defining database and collection
db = client["Regression"]
collection = db["household_power_consumption"]
#Fetching data from database
data = list(collection.find({},{'_id':0}))
df = pd.DataFrame(data)
df

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,month,total_metering
0,19/7/2007,09:28:00,0.210,0.084,228.56,1.0,0.0,0.0,0.0,7,0.0
1,15/12/2007,07:54:00,3.122,0.132,242.14,12.8,0.0,1.0,18.0,12,19.0
2,1/5/2009,05:22:00,1.830,0.458,242.12,7.8,0.0,1.0,18.0,5,19.0
3,2/5/2010,16:02:00,1.754,0.188,239.06,7.4,0.0,0.0,18.0,5,18.0
4,3/7/2010,23:52:00,1.272,0.250,241.00,5.4,0.0,0.0,1.0,7,1.0
...,...,...,...,...,...,...,...,...,...,...,...
14995,19/8/2010,06:49:00,0.000,0.000,0.00,0.0,0.0,0.0,0.0,8,0.0
14996,8/1/2009,21:22:00,2.266,0.066,241.81,9.4,0.0,0.0,18.0,1,18.0
14997,3/9/2009,20:47:00,0.524,0.062,238.84,2.2,0.0,0.0,0.0,9,0.0
14998,2/9/2010,12:33:00,1.378,0.150,237.61,5.8,0.0,1.0,18.0,9,19.0


### Removing features - Sub_metering1,Sub_metering2,Sub_metering3, date and time

In [3]:
df.drop(["Date","Time","Sub_metering_1","Sub_metering_2","Sub_metering_3"],axis=1,inplace=True)

In [4]:
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,month,total_metering
0,0.210,0.084,228.56,1.0,7,0.0
1,3.122,0.132,242.14,12.8,12,19.0
2,1.830,0.458,242.12,7.8,5,19.0
3,1.754,0.188,239.06,7.4,5,18.0
4,1.272,0.250,241.00,5.4,7,1.0


## DB Operations 

### Restoring new data in mongo db

In [5]:
## Deleting old data
db = client["Regression"]
if "household_power_consumption" in db.list_collection_names():
    db.drop_collection("household_power_consumption")
#defining collection
collection = db["household_power_consumption"]
#Storing data into database
res = collection.insert_many(json.loads(df.to_json(orient="records")))
print(res)